# NISQ Experiment

In [1]:
import sys
sys.path.insert(0, '../../src/')

import numpy as np
import qiskit as qk
import matplotlib.pyplot as plt
import multiprocessing as mp
import random
import pickle

from qiskit.quantum_info import DensityMatrix
from qiskit.quantum_info import Operator
from scipy.linalg import sqrtm
from tqdm.notebook import tqdm
from qiskit.providers.aer import AerSimulator

from cost_functions import *
from optimization import *
from quantum_maps import *
from quantum_tools import *
#np.set_printoptions(threshold=sys.maxsize)

In [18]:
#qk.IBMQ.save_account("66718f8f8aef22bcb6ebe86ad94a11f1fd1f4c55100829bb13f16e6b448e0a1ec6d09c459d738f58d0cbd8398a2a1f5e185a4706a61b6f896a5ce2983e136429", overwrite=True) 
provider = qk.IBMQ.load_account()
provider = qk.IBMQ.get_provider(hub='ibm-q', group='open', project='main')
backend = provider.get_backend("ibmq_santiago")

QiskitBackendNotFoundError: 'No backend matches the criteria'

## Quantum State Tomography

### Two Qubits

In [3]:
n = 2

circuit_target = qk.QuantumCircuit(n)
circuit_target.h(0)
circuit_target.s(1)
circuit_target.cnot(0,1)

basis_list = [prepare_input(numberToBase(i, 6, n), return_mode = "circuit") for i in range(6**n)]
state_input_list = [prepare_input(numberToBase(i, 6, n), return_mode = "density") for i in range(6**n)]

qsr_list = []
for basis in basis_list:
    circuit = qk.circuit.QuantumCircuit.compose(basis, circuit_target)
    qsr_list.append(StateTomography(circuit))
    
result_list = [qsr.run(backend, shots=20000).block_for_results() for qsr in tqdm(qsr_list)]
state_target_list = [np.array(result.analysis_results("state").value) for result in result_list]

pickle.dump([state_input_list, state_target_list], open("twoQubits_singleBlock.p", "wb"))

NameError: name 'prepare_input' is not defined

### Three Qubits

In [5]:
n = 3

circuit_target = qk.QuantumCircuit(n)
circuit_target.h(0)
circuit_target.s(1)
circuit_target.cnot(0,1)
circuit_target.h(1)
circuit_target.s(2)
circuit_target.cnot(1,2)

basis_list = [prepare_input(numberToBase(i, 6, n), return_mode = "circuit") for i in range(6**n)]
state_input_list = [prepare_input(numberToBase(i, 6, n), return_mode = "density") for i in range(6**n)]

qsr_list = []
for basis in basis_list:
    circuit = qk.circuit.QuantumCircuit.compose(basis, circuit_target)
    qsr_list.append(StateTomography(circuit))
    
result_list = [qsr.run(backend, shots=20000).block_for_results() for qsr in tqdm(qsr_list)]
state_target_list = [np.array(result.analysis_results("state").value) for result in result_list]

pickle.dump([state_input_list, state_target_list], open("threeQubits_singleBlock.p", "wb"))

## Pauli String Expectation Values

### Two Qubits

In [26]:
backend = AerSimulator()
n = 2
N = 1000
circuit_target = qk.QuantumCircuit(n)
circuit_target.h(0)
circuit_target.s(1)
circuit_target.cnot(0,1)
#circuit_target.h(1)
#circuit_target.s(2)
#circuit_target.cnot(1,2)

np.random.seed(42)
random.seed(42)

input_list = []
circuit_list = []
for i in range(N):
    
    index = np.random.randint(0, 6**n)
    config = numberToBase(index, 6, n)
    state = prepare_input(config)
    state_circuit = prepare_input(config, return_mode = "circuit")
    
    index = np.random.randint(0, 6**n)
    config = numberToBase(index, 6, n)
    observable = prepare_input(config)
    observable_circuit = prepare_input(config, return_mode = "circuit").inverse()
    
    input_list.append([state, observable])
    circuit = state_circuit
    circuit = circuit.compose(circuit_target)
    circuit = circuit.compose(observable_circuit)
    circuit.add_register(qk.ClassicalRegister(n))
    circuit.measure(circuit.qregs[0], circuit.cregs[0])
    
    circuit_list.append(circuit)

In [27]:
result_list = qk.execute(circuit_list, backend, shots = 10000).result()
counts_list = [result_list.get_counts(circuit) for circuit in circuit_list]

In [28]:
def expected_parity(counts):
    shots = sum(counts.values())
    parity = 0
    for string, count in counts.items():
        if string == n*"0":
            parity += count
        
    parity = parity/shots
    return parity

In [29]:
expectation_list = [expected_parity(counts) for counts in counts_list]
data = [input_list, expectation_list]

pickle.dump(data, open("..\..\data\twoQubits_expectation.p", "wb"))

In [30]:
expectation_list

[0.0,
 0.5139,
 0.2442,
 0.2417,
 0.2504,
 0.0,
 0.0,
 0.2539,
 0.0,
 0.4962,
 0.2521,
 0.4947,
 0.2511,
 0.2407,
 0.5027,
 0.0,
 0.244,
 0.4998,
 0.2547,
 0.2518,
 0.5004,
 0.2459,
 0.0,
 0.2459,
 0.2548,
 0.2562,
 0.2587,
 0.2578,
 0.2471,
 0.2529,
 0.0,
 0.4991,
 0.2531,
 0.2572,
 0.4953,
 0.0,
 0.2497,
 0.4973,
 0.5038,
 0.0,
 0.5015,
 0.4956,
 0.2491,
 0.2479,
 0.0,
 0.2478,
 0.2473,
 0.2452,
 0.2414,
 0.2459,
 0.2467,
 0.2497,
 0.2474,
 0.0,
 0.2503,
 0.0,
 0.2458,
 0.2469,
 0.4947,
 0.2516,
 0.4984,
 0.2508,
 0.2475,
 0.4985,
 0.2512,
 0.5013,
 0.5077,
 0.252,
 0.5013,
 0.25,
 0.252,
 0.2465,
 0.2469,
 0.0,
 0.2415,
 0.2534,
 0.5024,
 0.2454,
 0.0,
 0.2545,
 0.2478,
 0.2473,
 0.2599,
 0.4981,
 0.0,
 0.4983,
 1.0,
 0.2554,
 0.0,
 0.0,
 0.2472,
 0.2503,
 0.2521,
 0.5046,
 0.5083,
 0.2509,
 0.246,
 0.2499,
 0.2547,
 0.4962,
 0.4959,
 0.491,
 0.2502,
 0.2517,
 0.2539,
 0.2523,
 0.2431,
 0.5052,
 0.4941,
 0.0,
 0.2518,
 0.2495,
 0.2529,
 0.2518,
 0.0,
 0.4999,
 0.2482,
 0.5117,
 0.25